In [ ]:
# User Input
#image_path = './data/debug/easy_apple_small.jpg'
#mask_path = './data/debug/easy_apple_small_mask.jpg'
image_path = './data/object_enhancement/091_in.jpg'
mask_path = './data/object_enhancement/masks/091_mask.jpg'
desired_output_image_path = './data/object_enhancement/091_out.jpg'

delta_s = 0.8

# Constants
EPSILON = 1e-3

# Modular Definitions
compute_saliency_map = sm.tempsal_saliency
minimize_J = opt.minimize_J_global_poisson
compute_database = db.compute_location_database


In [ ]:
# Read the image
img = cv2.imread(image_path)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
print("Image size:", img.shape)

mask_image = cv2.imread(mask_path)
mask_image = cv2.cvtColor(mask_image, cv2.COLOR_BGR2GRAY)[:,:]
print("Mask size:", mask_image.shape)

#show desired output image
desired_output_image = cv2.imread(desired_output_image_path)
desired_output_image = cv2.cvtColor(desired_output_image, cv2.COLOR_BGR2RGB)




utils.display_images([img, mask_image,desired_output_image], titles=['Original Image', 'Mask','Desired Output Image'])

In [ ]:


mask_image_resized = cv2.resize(mask_image, (img.shape[1], img.shape[0]), interpolation=cv2.INTER_NEAREST)
assert img.shape[:2] == mask_image_resized.shape[:2], "Image and mask must match in size"

_, mask_image_resized = cv2.threshold(mask_image_resized, 127, 255, cv2.THRESH_BINARY)




In [ ]:
# === RUN BLOCK for Multiple Iterations with BFS Recursive PatchMatch (New Version) ===
PATCH_SIZE = 7
MAX_DEPTH = 999
run_times = 5

# Initialization
tau_positive = 0.3
tau_negative = 0.3
learning_rate_positive = 0.1
learning_rate_negative = 0.1
delta_s = 0.8

# Convert input image to Lab space
J_in = cv2.cvtColor(img, cv2.COLOR_RGB2Lab)
s_map = compute_saliency_map(J_in)

for i in range(run_times):
    print(f"\n--- Iteration {i+1} ---")

    # Compute D+ and D- databases and masks
    D_positive, D_negative, d_pos_mask, d_neg_mask = compute_database(
        tau_positive, tau_negative, J_in, s_map
    )

    # Run BFS Recursive PatchMatch with Poisson & border handling
    J_lab_result, vote_weights = minimize_J_global_poisson_bfs(
        J_in,
        mask_image_resized,
        D_positive,
        D_negative,
        d_pos_mask,
        d_neg_mask,
        patch_size=PATCH_SIZE,
        lambda_factor=0.5
    )

    # Convert to RGB for display
    J_rgb = cv2.cvtColor(J_lab_result, cv2.COLOR_Lab2RGB)

    # Show result
    plt.figure(figsize=(10, 4))
    plt.subplot(1, 2, 1)
    plt.imshow(J_rgb)
    plt.title(f'Patched Image - Iter {i+1}')
    plt.axis('off')

    plt.subplot(1, 2, 2)
    plt.imshow(s_map, cmap='hot')
    plt.title(f'Saliency Map - Iter {i+1}')
    plt.axis('off')
    plt.show()

    # Show current D+ and D- database patch visualizations
    I_D_positive, I_D_negative = db.compute_image_database(J_in, D_positive, D_negative)
    utils.display_images([I_D_positive, I_D_negative], ['D+', 'D-'])

    # Prepare for next iteration
    J_in = cv2.cvtColor(J_rgb.astype(np.uint8), cv2.COLOR_RGB2Lab)
    s_map = compute_saliency_map(J_in)

    # Saliency feedback update
    assert s_map.shape[:2] == mask_image_resized.shape, "Mismatch between saliency map and mask."
    saliency_foreground = np.mean(s_map[mask_image_resized > 0])
    saliency_background = np.mean(s_map[mask_image_resized == 0])
    current_contrast = saliency_foreground - saliency_background

    print(f"Saliency Foreground: {saliency_foreground:.4f}, Saliency Background: {saliency_background:.4f}")
    print(f"Current contrast: {current_contrast:.4f}, Desired contrast: {delta_s:.4f}")

    # Update tau values
    adjustment_positive = np.abs(current_contrast - delta_s) * learning_rate_positive
    adjustment_negative = np.abs(current_contrast - delta_s) * learning_rate_negative
    tau_positive = np.clip(tau_positive + adjustment_positive, 0, 1)
    tau_negative = np.clip(tau_negative - adjustment_negative, 0, 1)

    print(f"Updated tau_positive: {tau_positive:.4f}, tau_negative: {tau_negative:.4f}")

# Final output comparison
utils.display_images([img, J_rgb, desired_output_image],
                     ['Original Image', 'Final Image', 'Desired Output Image'])

# Saliency map comparison
s_map_original = compute_saliency_map(cv2.cvtColor(img, cv2.COLOR_RGB2Lab))
s_map_final = compute_saliency_map(cv2.cvtColor(J_rgb, cv2.COLOR_RGB2Lab))
s_map_desired = compute_saliency_map(cv2.cvtColor(desired_output_image, cv2.COLOR_RGB2Lab))

utils.display_images([s_map_original, s_map_final, s_map_desired],
                     ['Original Image Saliency Map', 'Final Image Saliency Map', 'Desired Output Image Saliency Map'])
